In [29]:
#import packages and create date

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import *
import shutil
import difflib
from pulp import *
import openpyxl
import glob

#import datetime
today = date.today()
#today = today - datetime.timedelta(days=1)
today = today.strftime("%m.%d.20%y")
today

time = "10pm_"

In [30]:
## DKNG Player Pool

contest = "PlayerPool_"
os.chdir(r"C:\Users\vchang\Downloads")

downloads = r'C:/Users/vchang/Downloads'
contest_data_loc = r'C:/Users/vchang/Documents/Fantasy Bball/DFS/DKNG_Data/PlayerPool'

df = pd.read_csv(r'C:/Users/vchang/Downloads/DKEntries.csv', header=7, usecols=[13,15,16,17,18,19,20,21])
df.to_csv("DKSalaries_"+ contest + str(today)+".csv", index=False)

shutil.move(downloads + "/"+ "DKSalaries_"+ contest + str(today)+".csv",
           contest_data_loc + "/"+ "DKSalaries_"+ contest + str(today)+".csv")

os.chdir(r"C:/Users/vchang/Documents/Fantasy Bball/DFS/DKNG_Data/PlayerPool")
# dkng_file = shutil.copy("DKSalaries.csv", "DKSalaries_"+ contest + str(today)+".csv")
dkng_file = pd.read_csv(r"C:/Users/vchang/Documents/Fantasy Bball/DFS/DKNG_Data/PlayerPool/DKSalaries_"+ contest + str(today)+".csv")
dkng_file.to_excel(r"C:/Users/vchang/Documents/Fantasy Bball/DFS/DKNG_Data/PlayerPool/DKSalaries_"+ contest + str(today)+".xlsx", index=False)

os.chdir(r"C:/Users/vchang/Documents/Fantasy Bball/DFS/DKNG_Data/PlayerPool")
os.remove("DKSalaries_"+ contest + str(today)+".csv")

In [31]:
## Uploads
## Uploads
os.chdir(r"C:\Users\vchang\Downloads")

draftkings_upload = r"C:\Users\vchang\Documents\Fantasy Bball\DFS\Upload\DraftKings"

dk_upload = glob.glob('DKEntries.csv')[0]
os.rename(dk_upload, "DK_Upload_"+ contest + str(today)+".csv")

shutil.move(downloads + "/"+ "DK_Upload_"+ contest + str(today)+".csv",
           draftkings_upload + "/"+ "DK_Upload_"+ contest + str(today)+".csv")

'C:\\Users\\vchang\\Documents\\Fantasy Bball\\DFS\\Upload\\DraftKings/DK_Upload_PlayerPool_03.18.2022.csv'

In [32]:
## ETR Daily

# changes the working directory to your downloads folder
os.chdir(r"C:\Users\vchang\Downloads")

downloads = r'C:/Users/vchang/Downloads'
data_loc = r'C:/Users/vchang/Documents/Fantasy Bball/DFS/ETR_Data/DraftKings'
daily_file = "DK NBA Projections.csv"

shutil.move(downloads + "/"+ daily_file,
           data_loc + "/"+ daily_file)

os.chdir(data_loc)

df = pd.read_csv(daily_file)

df.to_excel("ETR_Daily_"+str(today)+".xlsx", index=False)
os.remove("DK NBA Projections.csv")

In [33]:
## Joins the DKNG and ETR Files to include DKNG Positions 
    ## ETR does not include G/F/UTIL

os.chdir(r"C:\Users\vchang\Documents\Fantasy Bball\DFS\PlayerPool Analysis")
etr = pd.read_excel(r'C:\Users\vchang\Documents\Fantasy Bball\DFS\ETR_Data\DraftKings\ETR_Daily_' + str(today) + ".xlsx")
dkng = pd.read_excel(r'C:\Users\vchang\Documents\Fantasy Bball\DFS\DKNG_Data\PlayerPool\DKSalaries_PlayerPool_' + str(today) + ".xlsx")


dkng['ID'] = dkng['ID'].astype(str)
dkng['Name + ID'] = dkng['Name'] + ' (' + dkng['ID'] + ')'

## TO DO - check if merge on only DKNG Roster Position Works
df = pd.merge(etr, 
              dkng[['Name','Roster Position', 'Name + ID', 'Salary']], 
              left_on='Player', 
              right_on='Name')

In [34]:
## Unnests Roster Positions into each Row (explode in Python) ##


## Update Salary to INT 
df['DK Salary'] = df['DK Salary'].str.replace(",","").astype(float)

df['Roster Position'] = df['Roster Position'].astype(str)
df['Roster Position'] = df['Roster Position'].str.split('/')

df = df.explode('Roster Position')

df.head()

df.to_csv("DKNG_final_PlayerPool_ETR_"+str(today)+".csv", index=False)
# os.system("DKNG_final_PlayerPool_ETR_"+str(today)+".xlsx")

In [35]:
df.head()

,Player,Team,Opp,Minutes,DK Position,DK Salary,DK Points,DK Value,DK Ceiling,DK Own,DK Slate,Name,Roster Position,Name + ID,Salary
0,Jayson Tatum,BOS,SAC,37.5,SF/PF,9500.0,50.6,6.6,59.1,26.4,MAIN,Jayson Tatum,SF,Jayson Tatum (21734905),9500
0,Jayson Tatum,BOS,SAC,37.5,SF/PF,9500.0,50.6,6.6,59.1,26.4,MAIN,Jayson Tatum,PF,Jayson Tatum (21734905),9500
0,Jayson Tatum,BOS,SAC,37.5,SF/PF,9500.0,50.6,6.6,59.1,26.4,MAIN,Jayson Tatum,F,Jayson Tatum (21734905),9500
0,Jayson Tatum,BOS,SAC,37.5,SF/PF,9500.0,50.6,6.6,59.1,26.4,MAIN,Jayson Tatum,UTIL,Jayson Tatum (21734905),9500
1,Domantas Sabonis,SAC,BOS,34.0,C,8700.0,47.2,6.4,55.9,3.4,MAIN,Domantas Sabonis,C,Domantas Sabonis (21734858),8700


In [36]:
df = pd.read_csv("DKNG_final_PlayerPool_ETR_"+str(today)+".csv", index_col=['Name + ID', 'Roster Position'], skipinitialspace=True)

legal_assignments = df.index # tuples of (name, pos)
name_set = df.index.unique(0) # a convenience

costs = df['Salary'].to_dict()
values = df['DK Points'].to_dict()

# set up LP
draft = pulp.LpVariable.dicts('selected', legal_assignments, cat='Binary')

prob = pulp.LpProblem('the draft', LpMaximize)

#obj
prob += pulp.lpSum([draft[n,p]*values[n,p] for (n,p) in legal_assignments])

#salary cap
prob += pulp.lpSum([draft[n,p]*costs[n,p] for (n, p) in legal_assignments]) <= 50000

# positions
prob += pulp.lpSum([draft[n, p] for (n, p) in legal_assignments if p == 'PG']) == 1
prob += pulp.lpSum([draft[n, p] for (n, p) in legal_assignments if p == 'SG']) == 1
prob += pulp.lpSum([draft[n, p] for (n, p) in legal_assignments if p == 'G']) == 1
prob += pulp.lpSum([draft[n, p] for (n, p) in legal_assignments if p == 'SF']) == 1
prob += pulp.lpSum([draft[n, p] for (n, p) in legal_assignments if p == 'PF']) == 1
prob += pulp.lpSum([draft[n, p] for (n, p) in legal_assignments if p == 'F']) == 1
prob += pulp.lpSum([draft[n, p] for (n, p) in legal_assignments if p == 'C']) == 1
prob += pulp.lpSum([draft[n, p] for (n, p) in legal_assignments if p == 'UTIL']) == 1

# use each player at most only once
result = []
for name in name_set:
    prob += pulp.lpSum([draft[n, p] for (n, p) in legal_assignments if n == name]) <= 1

prob.solve()

for idx in draft:
    if draft[idx].varValue > 0:
        print(idx[0], idx[1])
        
        result.append([idx[0], idx[1]])

Jayson Tatum (21734905) SF
Domantas Sabonis (21734858) UTIL
Jaylen Brown (21734866) G
Davion Mitchell (21734948) PG
Mikal Bridges (21734852) F
Torrey Craig (21734750) PF
Justin Holiday (21734724) SG
Deandre Ayton (21734942) C


C:\Users\vchang\Anaconda3\lib\site-packages\pulp\pulp.py:1313: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


In [37]:
new_list = result
df = pd.DataFrame(new_list)
test = df[[1,0]]
test1 = test.transpose()
test2 = test1.rename(columns=test1.iloc[0])
test3 = test2.drop([1])
df2 = test3[['PG', 'SG','SF', 'PF', 'C', 'G', 'F', 'UTIL']]
df2.to_csv("dkng_result_" + str(today) + ".csv", index=None)
# os.system("dkng_result_" + str(today) + ".csv")

In [38]:
df2 = pd.read_csv(r'C:\Users\vchang\Documents\Fantasy Bball\DFS\PlayerPool Analysis\dkng_result_' + str(today) + ".csv")

upload = pd.read_csv(r'C:\Users\vchang\Documents\Fantasy Bball\DFS\Upload\DraftKings\DK_Upload_PlayerPool_' + str(today) + ".csv"
                      , error_bad_lines=False, engine ='python')

upload = upload[['Entry ID', 'Contest Name', 'Contest ID', 'Entry Fee']]

os.chdir(r'C:\Users\vchang\Documents\Fantasy Bball\DFS\Upload\DraftKings')
result = pd.concat([upload, df2], axis=1)
result.to_csv("DK_Final_Upload_"+ time + str(today)+".csv", index=False)
os.system("DK_Final_Upload_"+ time + str(today)+".csv")

Skipping line 8: Expected 14 fields in line 8, saw 22
Skipping line 9: Expected 14 fields in line 9, saw 22
Skipping line 10: Expected 14 fields in line 10, saw 22
Skipping line 11: Expected 14 fields in line 11, saw 22
Skipping line 12: Expected 14 fields in line 12, saw 22
Skipping line 13: Expected 14 fields in line 13, saw 22
Skipping line 14: Expected 14 fields in line 14, saw 22
Skipping line 15: Expected 14 fields in line 15, saw 22
Skipping line 16: Expected 14 fields in line 16, saw 22
Skipping line 17: Expected 14 fields in line 17, saw 22
Skipping line 18: Expected 14 fields in line 18, saw 22
Skipping line 19: Expected 14 fields in line 19, saw 22
Skipping line 20: Expected 14 fields in line 20, saw 22
Skipping line 21: Expected 14 fields in line 21, saw 22
Skipping line 22: Expected 14 fields in line 22, saw 22
Skipping line 23: Expected 14 fields in line 23, saw 22
Skipping line 24: Expected 14 fields in line 24, saw 22
Skipping line 25: Expected 14 fields in line 25, saw

0